In [1]:
pip install tensorflow 

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Train Data Generation 


In [5]:
batchsize = 8 

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=0.2, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

In [7]:
train_data = train_datagen.flow_from_directory(r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\mrlEyes_2018_01\Prepared_Data\train', target_size=(80,80), batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\mrlEyes_2018_01\Prepared_Data\train', target_size=(80,80), batch_size=batchsize, class_mode='categorical', subset='validation')

Found 61519 images belonging to 2 classes.
Found 15379 images belonging to 2 classes.


# Test Data Generation

In [19]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\mrlEyes_2018_01\Prepared_Data\test', target_size=(80,80), batch_size=8, class_mode='categorical')

Found 8000 images belonging to 2 classes.


# Import Base Model

In [9]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))

# Import higher Model

In [10]:
hmodel = bmodel.output

In [11]:
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs = hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [11]:
checkpoint = ModelCheckpoint(r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model\model.h5', monitor='val_loss', save_best_only = True, verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor = 'val_loss', patience=3, verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]

In [2]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model.fit(train_data, steps_per_epoch=train_data.samples//batchsize, validation_data = validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                   epochs = 50)

Epoch 1/50
7688/7689 [============================>.] - ETA: 0s - loss: 0.2094 - accuracy: 0.9188
Epoch 1: val_loss improved from inf to 0.21339, saving model to C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model\model.h5


C:\Users\Avinash\miniconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7689/7689 [==============================] - 418s 54ms/step - loss: 0.2093 - accuracy: 0.9188 - val_loss: 0.2134 - val_accuracy: 0.9106 - lr: 0.0010
Epoch 2/50
7689/7689 [==============================] - ETA: 0s - loss: 0.1804 - accuracy: 0.9304
Epoch 2: val_loss improved from 0.21339 to 0.18715, saving model to C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model\model.h5
7689/7689 [==============================] - 409s 53ms/step - loss: 0.1804 - accuracy: 0.9304 - val_loss: 0.1871 - val_accuracy: 0.9206 - lr: 0.0010
Epoch 3/50
7689/7689 [==============================] - ETA: 0s - loss: 0.1719 - accuracy: 0.9332
Epoch 3: val_loss did not improve from 0.18715
7689/7689 [==============================] - 416s 54ms/step - loss: 0.1719 - accuracy: 0.9332 - val_loss: 0.1975 - val_accuracy: 0.9216 - lr: 0.0010
Epoch 4/50
7689/7689 [==============================] - ETA: 0s - loss: 0.1631 - accuracy: 0.9372
Epoch 4: val_loss did not improve from 0.18715
7689/7689 [===========

# Model Evaluation

In [13]:
from tensorflow.keras.models import load_model
saved_model = load_model(r"C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model\model.h5")


In [20]:
test_accuracy = saved_model.evaluate(test_data)[1]
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

1000/1000 [==============================] - 37s 37ms/step - loss: 0.3465 - accuracy: 0.8992
Test Accuracy: 89.92%


In [26]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the saved model
saved_model = load_model(r"C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model\model.h5")

# Define the test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\mrlEyes_2018_01\Prepared_Data\test',
    target_size=(80, 80),
    batch_size=8,
    class_mode='categorical',
    shuffle=False  # Set shuffle to False to maintain order for the labels
)

# Use the model to make predictions on the test data
y_pred = saved_model.predict(test_data)

# Obtain the true labels from the directory structure
# The class labels are inferred from the subdirectories in your test directory
# You can use the class_indices attribute to map class labels to their corresponding subdirectories
true_labels = (test_data.class_indices.keys())

# Calculate confusion matrix and classification report
# Assuming you have true labels as subdirectories in your test directory
confusion = confusion_matrix(test_data.classes, y_pred.argmax(axis=1))
classification_rep = classification_report(test_data.classes, y_pred.argmax(axis=1), target_names=true_labels)

print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)


Found 8000 images belonging to 2 classes.
1000/1000 [==============================] - 42s 41ms/step
Confusion Matrix:
 [[3973   27]
 [ 779 3221]]
Classification Report:
               precision    recall  f1-score   support

  Close_Eyes       0.84      0.99      0.91      4000
   Open_Eyes       0.99      0.81      0.89      4000

    accuracy                           0.90      8000
   macro avg       0.91      0.90      0.90      8000
weighted avg       0.91      0.90      0.90      8000

